### GPS Antenna Delay




#### Antenna_cable 4 Setup

- The ImuBerry3 (192.168.1.72, RPi0) was generating 1 PPS signal fed into the stop pin of TDC7200 connected to RPi3 (192.168.1.68).
- The external GPS was generating 16MHz clock signal fed into the clock pins of TDC7200 and PIC12F675. The PIC divided the signal to 1 PPS which was connected to start pin of the TDC.
- GNSS configuration was same for both receivers - GPS, Galileo, Beidou, No SBAS, 0 cable and receiver delays, reported GPS time.

First half of measurement (tof 35-38 ns) was during the day. The tof is slowly rising. 

Later, at 1609019587,  the PIC 1 PPS signal was fed into ExtInt0 of RPi0. The change in layout?, manipulation of the cables?, led to tof drop to 24-26 ns and its stabilized.

#### Antenna_cable 5 Setup

- The ImuBerry3 (192.168.1.72, RPi0) was generating 6MHz signal fed into the stop pin of TDC7200 connected to RPi3 (192.168.1.68).
- The external GPS was generating 16MHz clock signal fed into the clock pins of TDC7200 and PIC12F675. The PIC divided the signal to 1 PPS which was connected to start pin of the TDC.
- GNSS configuration was same for both receivers - GPS, Galileo, Beidou, No SBAS, 0 cable and receiver delays, reported GPS time.
- TDC7200 was set to measurement mode 1 starting measurement avery 0.9s after the previous one (waiting 100ms for the start pulse)


#### Antenna_cable 6 Setup

- The ImuBerry3 (192.168.1.72, RPi0) was generating 3MHz signal fed into the stop pin of TDC7200 connected to RPi3 (192.168.1.68).
- The external GPS was generating 16MHz clock signal fed into the clock pins of TDC7200 and PIC12F675. The PIC divided the signal to 1 PPS which was connected to start pin of the TDC.
- GNSS configuration was same for both receivers - GPS, Galileo, Beidou, No SBAS, 0 cable and receiver delays, reported GPS time.
- TDC7200 was set to measurement mode 1 starting measurement avery 0.9s after the previous one (waiting 100ms for the start pulse)
- from 1609314456.179 the RPi0 is switched to external antenna. Before internal was used.


#### Antenna_cable 7 Setup

- The ImuBerry3 (192.168.1.72, RPi0) was generating 3MHz signal fed into the stop pin of TDC7200 connected to RPi3 (192.168.1.68).
- The external GPS was generating 16MHz clock signal fed into the clock pins of TDC7200 and PIC12F675. The PIC divided the signal to 1 PPS which was connected to start pin of the TDC.
- GNSS configuration was same for both receivers - GPS, Galileo, Beidou, No SBAS, 0 cable and receiver delays, reported GPS time.
- TDC7200 was set to measurement mode 1 starting measurement avery 0.9s after the previous one (waiting 100ms for the start pulse)
- RPi0 GPS antenna cable delay configured to 20ns.
- gps_comparison.csv is based on these data

#### Antenna_cable 8 Setup

- same as Antenna_cable 7 except the RPi0 GPS antenna cable delay set to 8ns

#### Antenna_cable 9 Setup

- same as Antenna_cable 7 except the RPi0 GPS antenna cable delay set to 28ns

#### Antenna_cable 10 Setup

- same as Antenna_cable 7 except the RPi0 GPS antenna cable delay set to 16ns
- logging (sync) to elasticsearch

#### Antenna_cable 11 Setup

- same as Antenna_cable 7 except the RPi0 GPS antenna cable delay set to 0ns
- logging (async) to elasticsearch

In [ ]:
elasticusernamepassword = '...'

In [2]:

import numpy as np
import pandas as pd
import matplotlib as mp
import allantools as at         # see https://github.com/aewallin/allantools
import datetime

In [13]:
fname = 'antenna_delay9'
name = fname + '-28ns'
filename = '/Users/dorsic/Documents/PhD/TDC7200/' + fname + '.tdc'
indexname = 'tdc'

def linetojson(line, clockMHz=3.0, name=None):
    align = 1.0/(clockMHz*2000000)
    lineparts = line.split(" ")
    ts = datetime.datetime.fromtimestamp(float(lineparts[0]))
    tof = float(lineparts[2][2:-2]) if (lineparts[2][2:-2] != 'None') else None
    tofa = None if not tof else (tof if tof < align else tof - 2*align)
    tofa = None if not tof else (tof+align if tof < align else tof)
    normLBS = float(lineparts[3][:-1])
    time1 = int(lineparts[4][1:-2])
    cal1 = int(lineparts[6][1:-1])
    cal2 = int(lineparts[7][:-2])
    res = {"_index": indexname, "_type": "_doc", "_source": {"@timestamp": ts, "tof": tof, "tofa": tofa, "normLBS": normLBS, "time1": time1, "cal1": cal1, "cal2": cal2, "name": name }}
    return res

def file_linecount(filename):
    with open(filename) as f:
        for i, l in enumerate(f):
            pass
    return i + 1


In [14]:
from elasticsearch import Elasticsearch, helpers

nlines = file_linecount(filename)
print("About to write {} records".format(nlines-16))

es = Elasticsearch(['https://' + elasticusernamepassword + '@rarach.northeurope.cloudapp.azure.com/elastic'])

res = es.delete_by_query(index='tdc',doc_type='_doc', body={"query": {"match": {"name": name}}})
print(res)

def log_file(filename):
    with open(filename, "r") as f:
        for i in range(16):
            f.readline()
        for line in f:
            yield linetojson(line, clockMHz=3, name=name)

res = helpers.bulk(es, log_file(filename))
print('DONE')

About to write 48879 records
{'took': 2103, 'timed_out': False, 'total': 27048, 'deleted': 27048, 'batches': 28, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []}
DONE


In [3]:
from elasticsearch import Elasticsearch, helpers
es = Elasticsearch(['https://' + elasticusernamepassword + '@rarach.northeurope.cloudapp.azure.com/elastic'])
res = es.delete_by_query(index='tdc',doc_type='_doc', body={"query": {"match": {"name": "ad11-0ns"}}})
print(res)

{'took': 282, 'timed_out': False, 'total': 5922, 'deleted': 5922, 'batches': 6, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []}


In [2]:
df = pd.read_csv("/Users/dorsic/Documents/PhD/TDC7200/gps_comparison.csv", sep=" ", )
align = 1.0/(3*2000000)
df.loc[df['TOF1'] < align, 'TOFA'] = df['TOF1']
df.loc[df['TOF1'] >= align, 'TOFA'] = df['TOF1'] - 2*align

In [5]:
from tqdm.notebook import tqdm
from sklearn.linear_model import LinearRegression

coefs = pd.DataFrame(columns=['coef', 'intercept'])
with tqdm(total=len(df)) as pbar:
    for i in range(2, len(df)):
        x = np.array(range(i)).reshape((-1, 1))
        y = np.array(df['TOFA'][0:i])
        model = LinearRegression().fit(x, y)
        coefs = coefs.append({'coef': model.coef_, 'intercept': model.intercept_}, ignore_index=True)
        pbar.update(1)

  0%|          | 0/187743 [00:00<?, ?it/s]

In [6]:

coefs.to_csv("/Users/dorsic/Documents/PhD/TDC7200/lin_regression.csv")